# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [13]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
main_document_tree = ET.parse( './data/mondial_database.xml' )

In [9]:
tree = ET.parse( './data/mondial_database.xml' )
root = tree.getroot()

In [15]:
# 1. 10 countries with the lowest infant mortality rates

#get country -> name -> infant mortality 

cl = []
imort = []
for element in tree.iterfind('country'):
    cl.append(element.find('name').text)
    try:
        imort.append(float(element.find('infant_mortality').text))
    except AttributeError:
        imort.append('none')
                     
df = pd.DataFrame(cl, columns=['Country'])
df['Infant_Mortality'] = imort
df.sort_values('Infant_Mortality').head(10)

,Country,Infant_Mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.6
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [16]:
# 2. 10 cities with the largest population

# country -> city -> population -> sort in descending order

city = []
pop = []
for element in tree.iterfind('country'):
    for subelement in element.findall('city'):
        city.append(subelement.find('name').text)
        try:
            pop.append(int(subelement.find("population[0]").text))
        except AttributeError:
            pop.append('none')
            
df = pd.DataFrame(city, columns=['City'])
df['Population'] = pop
df = df[df.Population != 'none']
df.sort_values('Population', ascending=False).head(10)

,City,Population
176,Seoul,9708483
164,Al Qahirah,8471859
80,Bangkok,7506700
128,Hong Kong,7055071
92,Ho Chi Minh,5968384
212,Singapore,5076700
163,Al Iskandariyah,4123869
216,New Taipei,3939305
177,Busan,3403135
107,Pyongyang,3255288


In [18]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

df = pd.DataFrame(columns=('Country', 'Population', 'Ethnic_Group', 'Percentage'))

i = 0

for element in tree.iterfind('country'):
    country = element.find('name').text
    pop = int(element.find("population[0]").text)
    for subelement in element.findall('ethnicgroup'):
        eg = subelement.text
        egp = float(subelement.get('percentage')) // 1
        df.loc[i] = [country, pop, eg, egp]
        i += 1

df['EG_Population'] = (df.Population * (df.Percentage / 100)) // 1
        
df.sort_values('EG_Population', ascending=False).head(10)

,Country,Population,Ethnic_Group,Percentage,EG_Population
176,China,1.360720e+09,Han Chinese,91.0,1.238255e+09
221,India,1.210855e+09,Indo-Aryan,72.0,8.718156e+08
220,India,1.210855e+09,Dravidian,25.0,3.027137e+08
345,United States,3.188571e+08,European,79.0,2.518971e+08
520,Nigeria,1.642945e+08,African,99.0,1.626516e+08
212,Bangladesh,1.497724e+08,Bengali,98.0,1.467769e+08
299,Japan,1.272980e+08,Japanese,99.0,1.260250e+08
93,Russia,1.436669e+08,Russian,79.0,1.134969e+08
278,Indonesia,2.521245e+08,Javanese,45.0,1.134560e+08
461,Brazil,2.027686e+08,European,53.0,1.074673e+08


In [22]:
# 4.

country = []
abbrev = []

for element in tree.iterfind('country'):
    country.append(element.find('name').text)
    abbrev.append(element.get('car_code'))
    
df = pd.DataFrame(country, columns=['Country'])
df['Abbrev'] = abbrev

In [23]:
# 4a. name & country of longest river
df1 = pd.DataFrame(columns=['River', 'Length', 'Abbrev'])

i = 0

for element in tree.iterfind('river'):
    river = (element.find('name').text)
    try:
        length = float((element.find('length').text))
    except AttributeError:
        length = 'none'
    for subelement in element.findall('located'):
        abbrev2 = subelement.get('country')
        df1.loc[i] = [river, length, abbrev2]
        i += 1

dfr = df1.merge(df, on='Abbrev')

dfr = dfr[dfr.Length != 'none']

dfr.sort_values('Length', ascending=False).head(3)

,River,Length,Abbrev,Country
258,Amazonas,6448,PE,Peru
250,Amazonas,6448,BR,Brazil
247,Amazonas,6448,CO,Colombia


In [25]:
# 4b. name and country of largest lake

df1 = pd.DataFrame(columns=['Lake', 'Area', 'Abbrev'])

i = 0

for element in tree.iterfind('lake'):
    lake = (element.find('name').text)
    try:
        area = float(element.find('area').text)
    except AttributeError:
        area = 'none'
    for subelement in element.findall('located'):
        abbrev2 = subelement.get('country')
        df1.loc[i] = [lake, area, abbrev2]
        i += 1
        
dfl = df1.merge(df, on='Abbrev')

dfl = dfl[dfl.Area != 'none']

dfl.sort_values(by='Area', ascending=False).head(4)

,Lake,Area,Abbrev,Country
67,Caspian Sea,386400,TM,Turkmenistan
64,Caspian Sea,386400,KAZ,Kazakhstan
63,Caspian Sea,386400,IR,Iran
50,Caspian Sea,386400,R,Russia


In [27]:
# 4c. name and country of airport at highest elevation

df1 = pd.DataFrame(columns=['Airport', 'Elevation', 'Abbrev'])

i = 0

for element in tree.iterfind('airport'):
    airport = (element.find('name').text)
    abbrev2 = element.get('country')
    try:
        elevation = float(element.find('elevation').text)
    except AttributeError:
        elevation = 'none'
    except TypeError:
        elevation = 'none'
    df1.loc[i] = [airport, elevation, abbrev2]
    i += 1
        
dfa = df1.merge(df, on='Abbrev')

dfa = dfa[dfa.Elevation != 'none']

dfa.sort_values(by='Elevation', ascending=False).head(1)

,Airport,Elevation,Abbrev,Country
80,El Alto Intl,4063,BOL,Bolivia
